In [2]:
from requests import get
from bs4 import BeautifulSoup
from pandas import DataFrame
import os
import re

In [3]:
containers = []
for link in ['https://www.pifers.com/auctions',"https://www.pifers.com/landlistings?page=0","https://www.pifers.com/landlistings?page=1","https://www.pifers.com/landlistings?page=2"]:
    response = get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    containers += soup.find_all('div', class_ = 'views-row')
print(len(containers))

147


In [14]:
class Item():
    def __init__(self, auctionID, legal, acres, cropAcres, pastureAcres, taxes, images, description) -> None:
        self.auctionID = auctionID
        self.description = description
        self.acres = acres
        self.legal = legal
        self.cropAcres = cropAcres
        self.pastureAcres = pastureAcres
        self.taxes = taxes
        self.images = images

    def __str__(self):
        return f"Auction ID: {self.auctionID}, Legal: {self.legal}, Acres: {self.acres},  Crop Acres: {self.cropAcres}, Pasture Acres: {self.pastureAcres}, Taxes: {self.taxes}, Images: {self.images}"
        
        
def landScrape(soup, data, auctionID):
    print("------------------------------------------------------------------------------------------")
    # print(soup)
    container = soup.find("div", class_="field-name-body")
    # print(container)
    plots = container.find_all("p")
    found = False
    empties = 0
    for i in range(len(plots)):
        if plots[i].text.strip()=="":empties+=1
        else:empties=0
        if empties == 2-(auctionID=="4954"):
            plots = plots[i+1:]
            break
    for i in range(len(plots)):
        for word in ["Parcel 1", "Parcel 1:", "Property Information", "Parcel Information"]:
            if word in plots[i].text:
                # data["Acres"].append([k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", plots[i].text) if k][0])
                plots = plots[i:]
                found = True
                break
        if found:break
    for i in plots:print(i.text)
    print("*"*60)
    while plots:
        legal = acres = cropAcres = pastureAcres = taxes = image = description = None
        empties = 0
        for i in range(len(plots)):
            p = plots[i]
            if "Legal" in p.text:legal = (p.text.replace("Legal:",""))
            elif "Cropland" in p.text or "Crop Acre" in p.text:
                val = [k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", p.text) if k]
                cropAcres = float(val[0].replace(",","")) if val else None
            elif "Pasture" in p.text or "Hay" in p.text:pastureAcres = (float([k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", p.text) if k][0].replace(",","")))
            elif "Taxes" in p.text:
                val = [k for k in re.findall("\d+[.,]?\d+[.,]?\d+", p.text) if k not in "2021 2022"]
                taxes = (float(val[0].replace(",","")) if val else None)
            elif len(p.find_all("img")) > 0:
                image = ("https://www.pifers.com"+p.img["src"])
            elif " Acres" in p.text.strip():pass
            elif "Acres:" in p.text:
                # plots = plots[i+1:]
                acres = (float([k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", p.text) if k][0].replace(",","")))
                # break
            elif plots[i].text.strip()=="":empties+=1
            else:empties = 0
            if empties == 2-(auctionID in ["4937", "4954"]) or i == len(plots)-1:
                plots = plots[i+1:]
                break

        obj = Item(auctionID, legal, acres, cropAcres, pastureAcres, taxes, image, description)
        print(obj)
        data.append(obj)

    # for i in range(parcels):
    #     # data["Names"].append(f"Parcel {i+1}")
    #     data["Auction IDs"].append(auctionID)
    #     data["Description"].append(None)
    #     empties = 0
    #     for j in range(len(plots)):
    #         p = plots[j]
    #         print(p.text)
    #         if "Legal" in p.text:data["Legal"].append(p.text.replace("Legal:",""))
    #         elif "Cropland" in p.text or "Crop Acre" in p.text:
    #             val = [k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", p.text) if k]
    #             data["Crop Acres"].append(float(val[0].replace(",","")) if val else None)
    #         elif "Pasture" in p.text or "Hay" in p.text:data["Pasture/Grass Acres"].append(float([k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", p.text) if k][0].replace(",","")))
    #         elif "Taxes" in p.text:
    #             val = [k for k in re.findall("\d+[.,]?\d+[.,]?\d+", p.text) if k not in "2021 2022"]
    #             data["Taxes($)"].append(float(val[0].replace(",","")) if val else None)
    #         elif len(p.find_all("img")) > 0:
    #             data["Images"].append("https://www.pifers.com"+p.img["src"])
    #         elif " Acres" in p.text.strip():pass
    #         elif "Acres:" in p.text:
    #             # plots = plots[i+1:]
    #             data["Acres"].append(float([k for k in re.findall("\d+[.,]?\d+[.,]?\d+||\d+", p.text) if k][0].replace(",","")))
    #             # break
    #         
    #         # if not(plots):break
    #     for entry in list(data.keys())[1:]:
    #         if len(data[entry]) < len(data["Names"]):data[entry].append(None)
    # # print(data)
    # for i in data:print(len(data[i]), i, data[i])

    

In [20]:
def itemScrape(soup, data, auctionID):
    items = soup.find_all("div", class_="auction-content-container")
    for item in items:
        data["Names"].append(item.find("h4").text)
        data["Auction IDs"].append(auctionID)
        for i in ["Acres", "Legal", "Crop Acres", "Pasture/Grass Acres", "Taxes($)"]:data[i].append(None)
        data["Description"].append(item.find("div", class_="stat-summary-description").text)
        data["Images"].append(item.find("img")["src"])

In [22]:
auctionData = {"Dates":[], "Prices": [], "IDs":[], "Locations":[], "Counts":[], "ItemTypes":[], "Parcels":[], "Categories":[], "Images":[], "Titles":[], "Details":[], "Realtors":[], "URLSlugs": []}
landData = {"Auction IDs": [], "Names": [], "Acres": [], "Legal": [], "Crop Acres": [], "Pasture/Grass Acres": [], "Taxes($)": [], "Description": [], "Images": []}
arr = []
for item in containers:
    t = "Item"
    if "acre" in item.find('div', class_="list-acres").text:t="Land"
    val = item.find('div', class_="list-date").text.replace("\n", '').strip()
    if "$" in val or "price" in val:
        auctionData["Dates"].append(None)
        auctionData["Prices"].append(val)
        t = "Real Estate"
    else:
        auctionData["Dates"].append(val)
        auctionData["Prices"].append(None)
    auctionData["IDs"].append(item.find('div', class_="id-number").text.replace("\n", '').strip())
    auctionData["Locations"].append(item.find('div', class_="list-city-state").text.replace("\n", '').strip())
    count = re.findall("\d+[.,]?\d+",item.find('div', class_="list-acres").text)
    auctionData["Counts"].append(float(count[0].replace(",","")) if count else None)
    auctionData["Categories"].append(item.find('div', class_="listing-category").text.replace("\n", '').strip())
    auctionData["ItemTypes"].append(t)
    parcels = re.findall("\d+",item.find('div', class_="parcel-count").text)[0]
    auctionData["Parcels"].append(int(parcels) if parcels else None)
    auctionData["Images"].append(item.find('img')["src"])
    auctionData["Titles"].append(item.find('div', class_="listing-title").text.replace("\n", '').strip())
    auctionData["Details"].append(item.find('div', class_="listing-details").text.replace("\n", '').strip())
    auctionData["Realtors"].append(item.find('div', class_="listing-realtor").text.replace("\n", '').strip())
    auctionData["URLSlugs"].append(item.find("a")["href"])
    
    try:
        response = get('https://www.pifers.com'+auctionData["URLSlugs"][-1])
        itemSoup = BeautifulSoup(response.text, 'html.parser')
    except:
        continue
    if auctionData["ItemTypes"][-1] == "Land":
        landScrape(itemSoup, arr, auctionData["IDs"][-1])
        pass
    else:
        # itemScrape(itemSoup, itemData)
        pass
for item in arr:
    if not(item.legal):continue
    landData["Auction IDs"].append(item.auctionID)
    landData["Acres"].append(item.acres)
    landData["Crop Acres"].append(item.cropAcres)
    landData["Description"].append(item.description)
    landData["Images"].append(item.images)
    landData["Pasture/Grass Acres"].append(item.pastureAcres)
    landData["Taxes($)"].append(item.taxes)
    landData["Legal"].append(item.legal)
    landData["Names"].append(None)

------------------------------------------------------------------------------------------
Parcel 1
Acres: 100.45 +/-
Legal: SE¼NW¼ & NE¼SW¼ & PT of Gov Lots 1 & 2 All lying East of Child’s Lake Rd 11-127-37
Crop Acres: 95.5 +/-
Taxes (2022): $2,140.00
Property ID #: 21-0196-100
This parcel features 100.45 acres with 95.5 +/- crop acres. Irrigated equipment is owned by the current tenant.  Well Permit # 796224.  Available for 2023 crop year! This parcel is subject to a right of first refusal.
 

 
Parcel 2
Acres: 119.82 +/-
Legal: S½SE¼ & SE¼SW¼ Lying East of CL Childs Lake Rd 11-127-37
Crop Acres: 112.99 +/-
Taxes (2022): $2,492.00
Property ID #: 21-0198-100
This parcel features 119.82 acres with 112.99 +/- crop acres. Irrigated equipment is owned by the current tenant.  Well Permit #796225.  Available for 2023 crop year! This parcel is subject to a right of first refusal.
 

 
Parcel 3:
Acres: 181.04 +/-
Legal: N½NE¼ & E½NW¼ & PT of Gov Lot 1 Lying E of Childs Lake Rd 14-127-37
Crop 

In [25]:
adf = DataFrame(auctionData)
display(adf)
adf.to_csv("piferAuction.csv")

,Dates,Prices,IDs,Locations,Counts,ItemTypes,Parcels,Categories,Images,Titles,Details,Realtors,URLSlugs
0,Thu Nov 10 10:00 AM CST,None,4913,"Alexandria, MN",401.30,Land,3,Land Auction,https://www.pifers.com/sites/default/files/sty...,"401.3 +/- Acres - Douglas County, MN",**CLICK HERE TO REGISTER & BID FREE ONLINE!!**...,Contact: Bob Pifer,/node/125635
1,Tue Nov 15 10:00 AM MST,None,4857,"Beach, ND",100.00,Item,1,Auction,https://www.pifers.com/sites/default/files/sty...,"Schaal Farm Retirement Auction - Beach, ND","Schaal Farm Retirement Auction Tuesday, Novemb...","Contacts: Andy Mrnak , Jim Sabe, John Soreide",https://pifers.nextlot.com/public/sales/99961/...
2,Wed Nov 16 11:00 AM CST,None,4916,"Upham, ND",640.00,Land,4,Land Auction,https://www.pifers.com/sites/default/files/sty...,"640 +/- Acres - McHenry County, ND",**CLICK HERE TO REGISTER & BID FREE ONLINE!!**...,Contact: Jack Pifer,/node/125716
3,Wed Nov 16 11:00 AM CST,None,4925,"Butte, ND",479.49,Land,3,Land Auction,https://www.pifers.com/sites/default/files/sty...,"479.49 +/- Acres - McLean & Sheridan Counties, ND",**CLICK HERE TO REGISTER & BID FREE ONLINE!!**...,Contact: Kevin Pifer,/node/125919
4,Wed Nov 16 5:00 PM CST,None,4930,"Coleharbor, ND",477.78,Land,3,Land Auction,https://www.pifers.com/sites/default/files/sty...,"477.78 +/- Acres - McLean County, ND",**CLICK HERE TO REGISTER & BID FREE ONLINE!!**...,Contact: Alan Butts,/node/126040
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,None,Call for price,3584,"Bowman, ND",NaN,Real Estate,2,Commercial/Development,https://www.pifers.com/sites/default/files/sty...,Commercial & Industrial Lots For Sale - Bowman...,Crossroads Business Park Commercial & Industri...,"Contacts: Ali Pierce, Andy Mrnak",/node/96545
143,None,"$89,900",3564,"Esmond, ND",35.00,Real Estate,1,Ag Land/Farm & Ranch,https://www.pifers.com/sites/default/files/sty...,Farmstead For Sale - 35 +/- Acres - Benson Cou...,FARMSTEAD FOR SALE HUNTING & FISHING CAMP!! 35...,Contact: John Stober,/node/96239
144,None,"$62,000",3490,"Bowman, ND",1650.00,Real Estate,1,Residential,https://www.pifers.com/sites/default/files/sty...,**SALE PENDING!!** House For Sale - 205 5th Av...,**SALE PENDING!** HOUSE FOR SALE 205 5 th Ave....,Contact: Ali Pierce,/node/95065
145,None,"$110,000",3295,"Linton, ND",6624.00,Real Estate,1,Commercial/Development,https://www.pifers.com/sites/default/files/sty...,"Office Building For Sale - Linton, ND",COMMERCIAL BUILDING FOR SALE 317 S Broadway - ...,Contact: Ken Schneider,/node/92728


In [23]:
itemData = {"Names": [], "Auction IDs": [],  "Acres": [], "Legal": [], "Crop Acres": [], "Pasture/Grass Acres": [], "Taxes($)": [], "Description": [], "Images": []}

for file in os.listdir("itemAuctions"):
    print(file)

    web = open(f"itemAuctions/{file}", "r")
    item_soup = BeautifulSoup(web.read(), 'html.parser')

    itemScrape(item_soup, landData, file.replace("auction","").replace(".txt", ""))
# idf = DataFrame(itemData)
# display(idf)

auction4723.txt
auction4719.txt
auction4948.txt
auction4717.txt
auction4924.txt
auction4926.txt
auction4857.txt
auction4922.txt
auction4921.txt


In [26]:
df = DataFrame(landData)
display(df)
df.to_csv("piferItems.csv")

,Auction IDs,Names,Acres,Legal,Crop Acres,Pasture/Grass Acres,Taxes($),Description,Images
0,4913,None,100.45,SE¼NW¼ & NE¼SW¼ & PT of Gov Lots 1 & 2 All ly...,95.50,NaN,2140.00,None,https://www.pifers.com/sites/default/files/Str...
1,4913,None,119.82,S½SE¼ & SE¼SW¼ Lying East of CL Childs Lake R...,112.99,NaN,2492.00,None,https://www.pifers.com/sites/default/files/Str...
2,4913,None,172.23,N½NE¼ & E½NW¼ & PT of Gov Lot 1 Lying E of Ch...,NaN,NaN,3786.00,None,https://www.pifers.com/sites/default/files/Str...
3,4916,None,160.00,"N½ SW¼, N½ SE¼, 29-159-78",158.25,NaN,1123.14,None,https://www.pifers.com/sites/default/files/Han...
4,4916,None,160.00,SE¼ 6-158-78,152.41,NaN,1228.22,None,https://www.pifers.com/sites/default/files/Han...
...,...,...,...,...,...,...,...,...,...
890,4921,Minnesota Vikings & New England Patriots Trip ...,NaN,None,NaN,NaN,NaN,"Trip for 2 Thursday, November 24,...",https://wbi.cdn.nextlot.com/retrievePhoto.html...
891,4921,Minnesota Twins Caleb Thielbar Jersey,NaN,None,NaN,NaN,NaN,"#56, Original Game-Worn Jersey, Stitched, *SIG...",https://wbi.cdn.nextlot.com/retrievePhoto.html...
892,4921,Minnesota Vikings Chad Greenway Jersey,NaN,None,NaN,NaN,NaN,Minnesota Viking's Linebacker Chad Greenway #5...,https://wbi.cdn.nextlot.com/retrievePhoto.html...
893,4921,Big Frig Indian Motorcycle Sioux Falls Cooler,NaN,None,NaN,NaN,NaN,"Cutting Board Divider, Left-Hand Drain",https://wbi.cdn.nextlot.com/retrievePhoto.html...
